# Pre-requisites


- WSL
- Miniconda3 

# Setup environment
- Create conda env `conda create langchain python=3.11`
- Set the "langchain" env that has been just created as the running env in VS code


Install langchain and openai package

In [ ]:
pip install langchain langchain-aws boto3 chromadb pypdf python-dotenv langchain-community

You need to set value of `AWS_ACCESS_KEY_ID`, `AWS_SECRET_ACCESS_KEY`, `AWS_REGION`  in the .env file

# Overviews
The BonBon FAQ.pdf file contains frequently asked questions and answers for customer support scenario. The topics are around IT related issue troubleshooting such as networking, software, hardware. You are requested to provide a solution to build a chat bot capable of answering the user questions with LangChain.

## Assignment 1: Document Indexing (mandatory)

- The content of BonBon FAQ.pdf should be indexed to the local Chroma vector DB from where the chatbot can lookup the appropriate information to answer questions.
- Should use some embedding model such as Azure Open AI text-embedding-ada-002 to create vectors, feel free to use any other open source embedding model if it works.

### Install Package

In [7]:

from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_aws import BedrockEmbeddings
from langchain.vectorstores import Chroma
from dotenv import load_dotenv
import boto3
import os

load_dotenv()

def vectorstore_faq_chroma(pdf_path: str = "data/BonBon FAQ.pdf", persist_dir: str = "chroma_db") -> Chroma:
    pdf_loader = PyPDFLoader(pdf_path)
    faq_documents = pdf_loader.load()

    bedrock_client = boto3.client(
        service_name='bedrock-runtime',
        region_name=os.getenv('AWS_REGION', 'us-east-1'),
        aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
        aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY')
    )

    embedding_model = BedrockEmbeddings(
        client=bedrock_client,
        model_id="amazon.titan-embed-text-v2:0"
    )

    vector_store = Chroma.from_documents(
        documents=faq_documents,
        embedding=embedding_model,
        persist_directory=persist_dir
    )
    
    return vector_store

vectorstore_faq_chroma()

## Assignment 2: Building Chatbot (mandatory)
- You are requested to build a chatbot solution for customer support scenario using Conversational ReAct agent supported in LangChain
- The chatbot is able to support user to answer FAQs in the sample BonBon FAQ.pdf file.
- The chatbot should use Azure Open AI GPT-3.5 LLM as the reasoning engine.
- The chatbot should be context aware, meaning that it should be able to chat with users in the conversation manner.
- The agent is equipped the following tools:
  - Internet Search: Help the chatbot automatically find out more about something using Duck Duck Go internet search
  - Knowledge Base Search: Help the chatbot to lookup information in the private knowledge base
- In case user asks for information related to topics in the BonBon FAQ.pdf file such as internet connection, printer, malware issues the chatbot must use the private knowledge base, otherwise it should search on the internet to answer the question.
- In the answer of chatbot, it should mention the source file and the page that the answer belongs to, for example the answer should mention "BonBon FQA.pdf (page 2)"

In [28]:
import os
import boto3
from typing import List
from langchain.agents import create_react_agent, AgentExecutor
from langchain.vectorstores import Chroma
from langchain.tools import Tool, DuckDuckGoSearchRun
from langchain_aws import BedrockEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain_aws import ChatBedrock
from langchain.prompts import PromptTemplate
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class BonBonChatbot:
    def __init__(self, pdf_path: str = "data/BonBon FAQ.pdf"):
        """
        Initialize the BonBon chatbot with AWS Bedrock Nova Lite and tools.
        """

        self.pdf_path = pdf_path
        self.knowledge_base = None
        
        self.session = boto3.Session()
        self.bedrock_client = self.session.client(
            service_name='bedrock-runtime',
            region_name=os.getenv('AWS_REGION', 'us-east-1'),
            aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
            aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY')
        )
        
        self.llm = ChatBedrock(
            client=self.bedrock_client,
            model_id="amazon.nova-lite-v1:0",
            model_kwargs={
                "max_tokens": 1000,
                "temperature": 0.7,
                "top_p": 0.9
            }
        )
        
        self.embeddings = BedrockEmbeddings(
            client=self.bedrock_client,
            model_id="amazon.titan-embed-text-v2:0"
        )
        
        self.setup_knowledge_base()
        self.tools = self.setup_tools()
        self.agent = self.setup_agent()
        
    def setup_knowledge_base(self):
        """Load and process the BonBon FAQ PDF into a vector store."""
        try:
            pdf_loader = PyPDFLoader(self.pdf_path)
            faq_documents = pdf_loader.load()
            self.knowledge_base = Chroma.from_documents(
                documents=faq_documents,
                embedding=self.embeddings,
                persist_directory="chroma_db"
            )
        except Exception as e:
            logger.error(f"Error setting up knowledge base: {str(e)}")
            raise
    
    def knowledge_base_search(self, query: str) -> str:
        """Search the knowledge base for relevant information."""
        try:
            if not self.knowledge_base:
                return "Knowledge base not available."
            
            docs = self.knowledge_base.similarity_search(query, k=3)
            
            if not docs:
                return "No relevant information found in the knowledge base."
            
            unique_docs = []
            response_parts = []
            seen_content = set()
            
            for doc in docs:
                content = doc.page_content.strip()
                if content not in seen_content:
                    seen_content.add(content)
                    unique_docs.append(doc)
            
            response_parts = []
            for doc in unique_docs:
                source = doc.metadata.get('source', 'Unknown')
                page = doc.metadata.get('page', 'Unknown')
                content = doc.page_content.strip()
                
                response_parts.append(f"From {source} (page {page}):\n{content}")
            
            return "\n\n".join(response_parts)
            
        except Exception as e:
            logger.error(f"Error in knowledge base search: {str(e)}")
            return "Error occurred while searching the knowledge base."
    
    def internet_search(self, query: str) -> str:
        """Search the internet using DuckDuckGo."""
        try:
            search_tool = DuckDuckGoSearchRun()
            results = search_tool.run(query)
            
            if not results:
                return "No internet search results found."
            
            return f"Internet search results for '{query}':\n{results}"
            
        except Exception as e:
            logger.error(f"Error in internet search: {str(e)}")
            return "Error occurred while searching the internet."
    
    def setup_tools(self) -> List[Tool]:
        """Setup the tools for the agent."""
        tools = [
            Tool(
                name="Knowledge Base Search",
                func=self.knowledge_base_search,
                description="""Use this tool to search for information in the BonBon FAQ knowledge base. 
                This tool should be used when users ask about:
                - Internet connectivity issues and troubleshooting
                - Printer problems and setup
                - Computer hardware and software issues
                - Malware removal and security
                - Network configuration
                - IT support procedures
                - Technical troubleshooting steps
                - Any other technical support questions that might be covered in the BonBon FAQ
                The tool will return information with source file and page number."""
            ),
            Tool(
                name="Internet Search",
                func=self.internet_search,
                description="""Use this tool to search for information on the internet using DuckDuckGo. 
                This tool should be used when:
                - The question is not related to BonBon FAQ topics
                - You need current information not available in the knowledge base
                - The user asks about general topics, news, or current events"""
            )
        ]
        
        return tools
    
    def setup_agent(self) -> AgentExecutor:
        """Setup the conversational React agent."""
        
        prompt_template = """You are a helpful customer support chatbot for BonBon. 
        You have access to tools that can help you answer questions.

        IMPORTANT INSTRUCTIONS:
        1. For questions about internet connection, printer issues, malware, or other technical support topics that might be in the BonBon FAQ, ALWAYS use the "Knowledge Base Search" tool first.
        2. For general questions, current events, or topics not related to BonBon FAQ, use the "Internet Search" tool.
        3. When providing answers from the knowledge base, always mention the source file and page number as provided by the tool.
        4. Be conversational and helpful in your responses.
        5. If you're unsure which tool to use, try the Knowledge Base Search first for technical questions.
        6. CRITICAL: Provide only ONE final answer. Do not repeat the question or provide multiple responses.
        7. Use the tool results to create a single, comprehensive answer.
        8. Do not include "Question:" or "Action:" in your final response - just provide the answer directly.

        You have access to the following tools:
        {tools}

        Use the following format:
        Question: the input question you must answer
        Action: the action to take, should be one of [{tool_names}]
        Action Input: the input to the action
        Observation: the result of the action
        Final Answer: the final answer to the original input question

        Question: {input}
        {agent_scratchpad}"""
        
        prompt = PromptTemplate(
            template=prompt_template,
            input_variables=["input", "agent_scratchpad", "tools", "tool_names"]
        )
        
        agent = create_react_agent(
            llm=self.llm,
            tools=self.tools,
            prompt=prompt
        )
        
        agent_executor = AgentExecutor(
            agent=agent,
            tools=self.tools,
            verbose=True,
            handle_parsing_errors=True,
            max_iterations=3,
            return_intermediate_steps=False 
        )
        
        return agent_executor
    
    def chat(self, user_input: str) -> str:
        """
        Process user input and return chatbot response.
        
        Args:
            user_input: User's message
            
        Returns:
            Chatbot's response
        """
        try:
            response = self.agent.invoke({"input": user_input})
            output =  response["output"]
        
            if "Final Answer:" in output:
                output = output.split("Final Answer:")[-1].strip()
            
            return output
        except Exception as e:
            logger.error(f"Error in chat: {str(e)}")
            return "I apologize, but I encountered an error processing your request. Please try again."

def main():
    """Main function to demonstrate the chatbot."""
    
    print("Initializing BonBon Chatbot...")
    chatbot = BonBonChatbot(
        pdf_path="data/BonBon FAQ.pdf"
    )
    
    print("BonBon Chatbot initialized successfully!")
    print("Type 'quit' to exit")
    print("-" * 50)
    
    while True:
        user_input = input("\nYou: ")
        
        if user_input.lower() == 'quit':
            print("Goodbye!")
            break

        print("\nBonBon Bot: ")
        response = chatbot.chat(user_input)
        print(response)
        print("-" * 50)

if __name__ == "__main__":
    # You can either run the interactive chat 
    main()

Initializing BonBon Chatbot...
BonBon Chatbot initialized successfully!
Type 'quit' to exit
--------------------------------------------------

BonBon Bot: 


> Entering new AgentExecutor chain...
Action: Knowledge Base Search
Action Input: I'm having trouble connecting to the internet, what should I do?
ObservationFrom data/BonBon FAQ.pdf (page 3):
2) Q: My internet connection is not working. Can you help me troubleshoot it? 
A: Please follow below steps: 
1) Check physical connections: 
• Ensure that all cables (Ethernet, modem, router, etc.) are securely connected. 
• Power cycle your modem and router by unplugging them from the power source, waiting for 30 
seconds, and then plugging them back in. 
 
2) Verify Wi-Fi settings (for wireless connections): 
• Make sure the Wi-Fi on your device is turned on. 
• Check if you are connected to the correct Wi-Fi network. 
• Try disconnecting and reconnecting to the Wi-Fi network. 
 
3) Test connectivity on other devices: 
• Check if other d

Internet search results for 'What is artificial intelligence?
Observation':
3 days ago · What is artificial intelligence? Artificial intelligence is the ability of a computer or computer-controlled robot to perform tasks … Mar 24, 2025 · Artificial Intelligence (AI) uses a wide range of techniques and approaches that enable machines to simulate human-like … Jun 27, 2025 · Artificial Intelligence or AI refers to machines that mimic human intelligence – things like problem-solving, pattern … Feb 17, 2025 · Discover the fascinating world of Artificial Intelligence in this extensive guide. Uncover how AI operates in a simplified and … Mar 5, 2025 · Artificial intelligence (AI) refers to the field of computer science which aims to make computer systems think, reason, learn, …Final Answer: Artificial intelligence (AI) refers to the field of computer science that aims to create systems capable of performing tasks that typically require human intelligence, such as problem-solving, learning, an

## Assignment 3: Build a new assistant based on BonBon source code (optional)
The objective
- Run the code and index the sample BonBon FAQ.pdf file to Azure Cognitive Search
- Explore the code and implement a new assistant that has the same behavior as above
- Explore other features such as RBACs, features on admin portal

Please contact the training team in case you need to get the source code of BonBon.